<a href="https://colab.research.google.com/github/nosadchiy/public/blob/main/getExchangeRates_Frankfurter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import requests
from datetime import datetime, timedelta, date
import pandas as pd
from scipy.stats import multivariate_normal
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum
from tabulate import tabulate
import datetime as dt
_empty_series = pd.Series(dtype=float)

In [51]:
import requests
from datetime import datetime, timedelta, date

def get_month_end_dates(start_date_str, end_date_str):
    """
    Generate a list of dates representing the last day of each month
    between start_date and end_date. If the end_date falls in the middle
    of a month, that date will be used for the final month.
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d').date()
    end_date   = datetime.strptime(end_date_str, '%Y-%m-%d').date()
    month_end_dates = []

    # Start from the first day of the month of start_date.
    current = start_date.replace(day=1)

    while current <= end_date:
        # Determine the first day of the next month.
        if current.month == 12:
            next_month = date(current.year + 1, 1, 1)
        else:
            next_month = date(current.year, current.month + 1, 1)

        # Last day of the current month is one day before the first day of the next month.
        last_day_of_month = next_month - timedelta(days=1)

        # If the last day of the month is after the specified end_date,
        # then use end_date (if it is in the same month) instead.
        if last_day_of_month > end_date:
            last_day_of_month = end_date

        if last_day_of_month >= start_date:
            month_end_dates.append(last_day_of_month)

        # Move to the first day of the next month.
        current = next_month

    return month_end_dates

def get_exchange_rate_for_date(date_str, base_currency="USD", symbols=None):
    """
    Fetches exchange rates for the given date from the Frankfurter API.

    :param date_str: The date string in 'YYYY-MM-DD' format.
    :param base_currency: The base currency (default "USD").
    :param symbols: A list of target currency codes.
    :return: A dictionary of exchange rates.
    """
    url = f"https://api.frankfurter.app/{date_str}"
    params = {"from": base_currency}
    if symbols:
        params["to"] = ",".join(symbols)

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data.get("rates", {})
    else:
        print(f"Error fetching data for {date_str}: HTTP {response.status_code}")
        return {}

def estimate_statistics(exchange_data):
    """
    Given a dictionary of exchange rate data with dates as keys and
    rate dictionaries as values, this function computes:

      - The average exchange rate vector.
      - The variance-covariance matrix of the exchange rates.

    :param exchange_data: dict of the form {date_str: {currency: rate, ...}, ...}
    :return: A tuple (average_vector, covariance_matrix) where:
             - average_vector is a pandas Series.
             - covariance_matrix is a pandas DataFrame.
    """
    # Convert the nested dictionary to a pandas DataFrame.
    # Rows are indexed by date and columns are the currency codes.
    df = pd.DataFrame.from_dict(exchange_data, orient="index")
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)

    # Calculate the average exchange rate for each currency.
    average_vector = df.mean()

    # Calculate the variance-covariance matrix of the exchange rates.
    covariance_matrix = df.cov()

    return average_vector, covariance_matrix

In [5]:
# Specify the time period and target currencies.
start_date_str = "2019-01-01"  # Change to your desired start date.
end_date_str   = "2023-12-31"  # Change to your desired end date.
target_currencies = ['BRL', 'EUR', 'INR', 'JPY', 'MXN']

# Get a list of month-end dates for the given period.
month_end_dates = get_month_end_dates(start_date_str, end_date_str)

# Dictionary to hold the results.
exchange_data = {}

# Fetch the exchange rates for each month-end date.
for d in month_end_dates:
    date_str = d.isoformat()
    print(f"Fetching rates for {date_str}...")
    rates = get_exchange_rate_for_date(date_str, base_currency="USD", symbols=target_currencies)
    exchange_data[date_str] = rates

# Display the collected exchange rate data.
print("\nExchange Rates at Month End:")
for date_str, rates in exchange_data.items():
    print(f"{date_str}: {rates}")

# Estimate average exchange rate vector and the variance-covariance matrix.
avg_vector, cov_matrix = estimate_statistics(exchange_data)

print("\nAverage Exchange Rate Vector:")
print(avg_vector)

print("\nVariance-Covariance Matrix:")
print(cov_matrix)

Fetching rates for 2019-01-31...
Fetching rates for 2019-02-28...
Fetching rates for 2019-03-31...
Fetching rates for 2019-04-30...
Fetching rates for 2019-05-31...
Fetching rates for 2019-06-30...
Fetching rates for 2019-07-31...
Fetching rates for 2019-08-31...
Fetching rates for 2019-09-30...
Fetching rates for 2019-10-31...
Fetching rates for 2019-11-30...
Fetching rates for 2019-12-31...
Fetching rates for 2020-01-31...
Fetching rates for 2020-02-29...
Fetching rates for 2020-03-31...
Fetching rates for 2020-04-30...
Fetching rates for 2020-05-31...
Fetching rates for 2020-06-30...
Fetching rates for 2020-07-31...
Fetching rates for 2020-08-31...
Fetching rates for 2020-09-30...
Fetching rates for 2020-10-31...
Fetching rates for 2020-11-30...
Fetching rates for 2020-12-31...
Fetching rates for 2021-01-31...
Fetching rates for 2021-02-28...
Fetching rates for 2021-03-31...
Fetching rates for 2021-04-30...
Fetching rates for 2021-05-31...
Fetching rates for 2021-06-30...
Fetching r

In [7]:
def get_distribution(start_date_str, end_date_str, target_currencies):
    # Get a list of month-end dates for the given period.
    month_end_dates = get_month_end_dates(start_date_str, end_date_str)
    # Dictionary to hold the results.
    exchange_data = {}
    # Fetch the exchange rates for each month-end date.
    for d in month_end_dates:
        date_str = d.isoformat()
        print(f"Fetching rates for {date_str}...")
        rates = get_exchange_rate_for_date(date_str, base_currency="USD", symbols=target_currencies)
        exchange_data[date_str] = rates
    avg_vector, cov_matrix = estimate_statistics(exchange_data)
    return avg_factor,cov_matrix

In [123]:
def get_distribution(start_date_str, end_date_str, target_currencies):
    # Get a list of month-end dates for the given period.
    month_end_dates = get_month_end_dates(start_date_str, end_date_str)
    # Dictionary to hold the results.
    exchange_data = {}
    # Fetch the exchange rates for each month-end date.
    for d in month_end_dates:
        date_str = d.isoformat()
        print(f"Fetching rates for {date_str}...")
        rates = get_exchange_rate_for_date(date_str, base_currency="USD", symbols=target_currencies)
        exchange_data[date_str] = rates
    avg_factor, cov_matrix = estimate_statistics(exchange_data)
    return avg_factor,cov_matrix
    
from tqdm import tqdm

year_list = ['2018','2019','2020','2021','2022','2023']
target_currencies = ['BRL', 'EUR', 'INR', 'JPY', 'MXN']
singal_year_dict = {year: None for year in year_list}
for i in tqdm(range(len(year_list))):
    start, end = year_list[i]+'-01-01', year_list[i]+'-12-31'
    avg_factor, cov_matrix = get_distribution(start, end, target_currencies)
    mvn_distribution = multivariate_normal(mean=avg_vector, cov=cov_matrix)
    sample = mvn_distribution.rvs(size=100, random_state = 40)
    # 数组末尾+1 代表USD
    sample_usd = np.hstack((sample, np.ones((sample.shape[0], 1))))
    singal_year_dict[year_list[i]] = sample_usd

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Fetching rates for 2018-01-31...
Fetching rates for 2018-02-28...
Fetching rates for 2018-03-31...
Fetching rates for 2018-04-30...
Fetching rates for 2018-05-31...
Fetching rates for 2018-06-30...
Fetching rates for 2018-07-31...
Fetching rates for 2018-08-31...
Fetching rates for 2018-09-30...
Fetching rates for 2018-10-31...
Fetching rates for 2018-11-30...
Fetching rates for 2018-12-31...


 17%|██████████████                                                                      | 1/6 [00:00<00:04,  1.03it/s]

Fetching rates for 2019-01-31...
Fetching rates for 2019-02-28...
Fetching rates for 2019-03-31...
Fetching rates for 2019-04-30...
Fetching rates for 2019-05-31...
Fetching rates for 2019-06-30...
Fetching rates for 2019-07-31...
Fetching rates for 2019-08-31...
Fetching rates for 2019-09-30...
Fetching rates for 2019-10-31...
Fetching rates for 2019-11-30...


 33%|████████████████████████████                                                        | 2/6 [00:01<00:03,  1.14it/s]

...

Fetching rates for 2022-01-31...
Fetching rates for 2022-02-28...
Fetching rates for 2022-03-31...
Fetching rates for 2022-04-30...
Fetching rates for 2022-05-31...
Fetching rates for 2022-06-30...
Fetching rates for 2022-07-31...
Fetching rates for 2022-08-31...
Fetching rates for 2022-09-30...
Fetching rates for 2022-10-31...
Fetching rates for 2022-11-30...
Fetching rates for 2022-12-31...


 83%|██████████████████████████████████████████████████████████████████████              | 5/6 [00:04<00:00,  1.07it/s]

Fetching rates for 2023-01-31...
Fetching rates for 2023-02-28...
Fetching rates for 2023-03-31...
Fetching rates for 2023-04-30...
Fetching rates for 2023-05-31...
Fetching rates for 2023-06-30...
Fetching rates for 2023-07-31...
Fetching rates for 2023-08-31...
Fetching rates for 2023-09-30...
Fetching rates for 2023-10-31...


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:05<00:00,  1.08it/s]

Fetching rates for 2023-11-30...
Fetching rates for 2023-12-31...


In [125]:
singal_year_dict['2023'][1]

array([  4.66418863,   0.86380228,  75.32200947, 116.74581847,
        18.76913779,   1.        ])

In [127]:
# 合成exrate0 不要像他那样dict和df用同一变量 容易出问题
index_list = [i for i in range(0,100,1)]
all_exrate0 = {index:pd.DataFrame() for index in index_list}
yl = ['2018', '2019', '2020', '2021', '2022', '2023']  # 确保 2023 包含在内
for index in index_list:
    exrate0_dict = {year: None for year in yl}
    for year in yl:  # 这里不再使用 yl[:-1]，确保 2023 也被包含
        try:
            exrate0_dict[year] = singal_year_dict[year][index]
        except:
            print(f'Error {year}-{index}')
    exrate0 = pd.DataFrame(exrate0_dict, index=['BRL', 'EUR', 'INR', 'JPY', 'MXN', 'USD'])
    all_exrate0[index] = exrate0

In [129]:
all_exrate0.items()

dict_items([(0,            2018        2019        2020        2021        2022        2023
BRL    4.924720    4.815209    4.840171    4.981084    4.982369    4.948563
EUR    0.892035    0.898454    0.942994    0.920696    0.931230    0.899112
INR   77.936970   76.392883   76.966143   76.498522   77.616620   75.966407
JPY  121.071014  118.969912  120.327470  121.585300  126.193120  115.865255
MXN   19.648274   19.731181   20.541614   19.599878   19.378984   20.039947
USD    1.000000    1.000000    1.000000    1.000000    1.000000    1.000000), (1,            2018        2019        2020        2021        2022        2023
BRL    4.836944    4.801779    5.634420    5.028104    4.947519    4.664189
EUR    0.914788    0.888853    0.820368    0.889318    0.926656    0.863802
INR   75.916290   74.038883   77.269746   74.731561   74.125984   75.322009
JPY  122.852661  118.445467  115.672928  121.476839  125.473381  116.745818
MXN   20.334837   19.963783   21.029716   19.269413   20.282895   

In [131]:
selected_yr = 2023
base_yr = 2019

demand = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'd_h': [  7, 15,  5,  7,  3, 18],
    'd_r': [  7, 12,  3,  8,  3, 17],
})
demand.set_index('from', inplace=True)

caps = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'cap': [18, 45, 18, 10, 30, 22],
})
caps.set_index('plant', inplace=True)

pcosts = pd.DataFrame({
    'plant': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'fc_p': [20, 45, 14, 13, 30, 23],
    'fc_h': [ 5, 13,  3,  4,  6,  5],
    'fc_r': [ 5, 13,  3,  4,  6,  5],
    'rm_h': [3.6, 3.9, 3.6, 3.9, 3.6, 3.6],
    'pc_h': [5.1, 6.0, 4.5, 6.0, 5.0, 5.0],
    'rm_r': [4.6, 5.0, 4.5, 5.1, 4.6, 4.5],
    'pc_r': [6.6, 7.0, 6.0, 7.0, 6.5, 6.5],
})
pcosts.set_index('plant', inplace=True)

tcosts = pd.DataFrame({
    'from': ['Brazil', 'Germany', 'India', 'Japan', 'Mexico', 'U.S.'],
    'LatinAmerica': [ 0.20, 0.45, 0.50, 0.50, 0.40, 0.45],
    'Europe':       [ 0.45, 0.20, 0.35, 0.40, 0.30, 0.30],
    'AsiaWoJapan':  [ 0.50, 0.35, 0.20, 0.30, 0.50, 0.45],
    'Japan':        [ 0.50, 0.40, 0.30, 0.10, 0.45, 0.45],
    'Mexico':       [ 0.40, 0.30, 0.50, 0.45, 0.20, 0.25],
    'U.S.':           [ 0.45, 0.30, 0.45, 0.45, 0.25, 0.20],
})
tcosts.set_index('from', inplace=True)

duties = pd.DataFrame({
    'from': ['LatinAmerica', 'Europe', 'AsiaWoJapan', 'Japan', 'Mexico', 'U.S.'],
    'duty': [ 0.30, 0.03, 0.27, 0.06, 0.35, 0.04],
})
duties.set_index('from', inplace=True)

In [133]:
# identify number of supply and demand location for iterations
n_ctry = range(demand.shape[0])
n_lines = range(demand.shape[1]+1)

# Objective function to calculate cost
def calc_total_cost(dec_plant, dec_h, dec_r, base_yr=2019, selected_yr=2023, tariff=0):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    reindx = exrate.loc[:, f'{base_yr}'] / exrate.loc[:, f'{selected_yr}']

    pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
    # return pcosts_rev
    # pcosts_rev = adj_pcosts_exrate(2019, 2023)

    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost


# Calculate excess capacity given decision variables
def calc_excess_cap(dec_plant, dec_h, dec_r):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    excess_cap = (x_plant * caps.values).copy()
    excess_cap[:, 0] -= (np.sum(x_h, axis=1) + np.sum(x_r, axis=1))
    excess_cap[:, 1] -= np.sum(x_h, axis=1)
    excess_cap[:, 2] -= np.sum(x_r, axis=1)
    return excess_cap

# Calculate unmet demand given decision variables
def calc_unmet_demand(dec_h, dec_r):
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    x_h_sum = np.sum(x_h, axis=0)
    x_r_sum = np.sum(x_r, axis=0)
    unmet_demand = (demand.values).copy()
    unmet_demand = np.column_stack((x_h_sum - unmet_demand[:, 0], x_r_sum - unmet_demand[:, 1]))

    return unmet_demand

In [135]:
# Prompt the user to enter the year
def opt_main_func1(exrate, selected_yr, tariff):
    
    # Create a Gurobi model
    model = Model("MinimizeCost")
    
    # Assign initial value of decision variables
    dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}
    
    # Define decision variables
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}
    
    # Excess Capacity constraints
    excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
    for i in n_ctry:
        for j in n_lines:
            model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")
    
    
    # Unmet demand constraints
    unnmet_demand = calc_unmet_demand(dec_h, dec_r)
    for i in n_ctry:
        for j in range(2):
            model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")
    
    
    # Update the model
    model.update()
    
    # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
    model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, base_yr, selected_yr, tariff), GRB.MINIMIZE)
    
    # Suppress optimization output
    model.Params.OutputFlag = 0
    
    # Optimize the model
    model.optimize()
    
    # Extract results to print as table
    op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)
    op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
    op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

    '''
    print("\nHighCal Flow\n")
    print(tabulate(op_h, headers='keys', tablefmt='pretty'))
    print("\nRelax Flow\n")
    print(tabulate(op_r, headers='keys', tablefmt='pretty'))
    print("\nStrategy\n")
    print(tabulate(op_plant, headers='keys', tablefmt='pretty'))
    print(f"\nMinimum Cost: $ {round(model.objVal,2)} in year {selected_yr} at Tariff {(tariff*100)}")
    '''
    
    #results = {
    #    "HighCal Flow": tabulate(op_h, headers='keys', tablefmt='pretty'),
    #    "Relax Flow": tabulate(op_r, headers='keys', tablefmt='pretty'),
    #    "Strategy": tabulate(op_plant, headers='keys', tablefmt='pretty'),
    #    "Minimum Cost": f"$ {round(model.objVal, 2)} in year {selected_yr} at Tariff {tariff * 100}%"
    #}

    return op_plant

# Q1 & 2

In [137]:
for id, exrate0 in tqdm(all_exrate0.items()):
    #print(f'solution for id:{id}')
    opplant = opt_main_func1(exrate0,2018,0.5) #changable
    res_dict[id] = opplant

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 31.92it/s]


In [139]:
res_dict[3]

,Plant,H,R
plant,,,
Brazil,1.0,1.0,1.0
Germany,1.0,0.0,1.0
India,1.0,1.0,1.0
Japan,0.0,-0.0,0.0
Mexico,1.0,1.0,1.0
U.S.,1.0,1.0,1.0


In [141]:
sum(res_dict[id] for id in all_exrate0.keys())/100

,Plant,H,R
plant,,,
Brazil,1.0,1.0,1.0
Germany,1.0,0.0,1.0
India,1.0,1.0,1.0
Japan,0.0,0.0,0.0
Mexico,1.0,1.0,1.0
U.S.,1.0,1.0,1.0


In [143]:
solution_year = [2018,2019,2020,2021,2022]
res_dict1 = {year:{id:pd.DataFrame() for id in all_exrate0.keys()} for year in solution_year}

for id, exrate0 in tqdm(all_exrate0.items()):
    #print(f'solution for id:{id}')
    for year in solution_year:
        opplant = opt_main_func1(exrate0,year,0.05)
        res_dict1[year][id] = opplant

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:16<00:00,  6.08it/s]


In [145]:
sum(sum(res_dict1[key].values()) for key in solution_year)/500

,Plant,H,R
plant,,,
Brazil,1.0,1.0,1.0
Germany,1.0,0.0,1.0
India,1.0,1.0,1.0
Japan,0.0,0.0,0.0
Mexico,1.0,1.0,1.0
U.S.,1.0,1.0,1.0


In [207]:
# Prompt the user to enter the year
def opt_main_func2(exrate, selected_yr, tariff):
    
    # Create a Gurobi model
    model = Model("MinimizeCost")
    
    # Assign initial value of decision variables
    dec_plant = {(i, j): 1 for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): 1 for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): 1 for i in n_ctry for j in n_ctry}
    
    # Define decision variables
    dec_plant = {(i, j): model.addVar(vtype=GRB.BINARY, name=f"Dec_plant_{i}_{j}")    for i in n_ctry for j in n_lines}
    dec_h     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_h_{i}_{j}") for i in n_ctry for j in n_ctry}
    dec_r     = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, lb=0, name=f"Dec_r_{i}_{j}") for i in n_ctry for j in n_ctry}
    
    # Excess Capacity constraints
    excess_cap = calc_excess_cap(dec_plant, dec_h, dec_r)
    for i in n_ctry:
        for j in n_lines:
            model.addConstr(excess_cap[i, j] >= 0, name=f"Excess_Cap_Constraints_{i}_{j}")
    
    
    # Unmet demand constraints
    unnmet_demand = calc_unmet_demand(dec_h, dec_r)
    for i in n_ctry:
        for j in range(2):
            model.addConstr(unnmet_demand[i,j] == 0, name=f"Unmet_Demand_Constraints_{i}_{j}")
    
    
    # Update the model
    model.update()
    
    # Set objective function - Total cost = Fixed cost + Variable costs of Highcal and Relax lines
    model.setObjective(calc_total_cost(dec_plant, dec_h, dec_r, base_yr, selected_yr, tariff), GRB.MINIMIZE)
    
    # Suppress optimization output
    model.Params.OutputFlag = 0
    
    # Optimize the model
    model.optimize()
    
    # Extract results to print as table
    op_plant = pd.DataFrame([[dec_plant[i, j].x for j in n_lines] for i in n_ctry], columns = ['Plant','H','R'], index=caps.index)
    op_h     = pd.DataFrame([[dec_h[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)
    op_r     = pd.DataFrame([[dec_r[i, j].x for j in n_ctry] for i in n_ctry], columns = tcosts.columns, index=tcosts.index)

    return op_h, op_r

In [209]:
res_h_dict = {}
res_r_dict = {}

for id, exrate0 in tqdm(all_exrate0.items()):
    #print(f'solution for id:{id}')
    oph, opr = opt_main_func2(exrate0,2019,0)
    res_h_dict[id] = oph
    res_r_dict[id] = opr

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 29.01it/s]


In [211]:
res_h_dict[2]

,LatinAmerica,Europe,AsiaWoJapan,Japan,Mexico,U.S.
from,,,,,,
Brazil,7.0,4.0,0.0,0.0,0.0,0.0
Germany,0.0,0.0,0.0,0.0,0.0,0.0
India,0.0,0.0,5.0,7.0,0.0,0.0
Japan,0.0,0.0,0.0,0.0,0.0,0.0
Mexico,0.0,11.0,0.0,0.0,3.0,13.0
U.S.,0.0,0.0,0.0,0.0,0.0,5.0


In [107]:
res_r_dict[2]

,LatinAmerica,Europe,AsiaWoJapan,Japan,Mexico,U.S.
from,,,,,,
Brazil,7.0,0.0,0.0,0.0,0.0,0.0
Germany,0.0,12.0,0.0,5.0,0.0,0.0
India,0.0,0.0,3.0,3.0,0.0,0.0
Japan,0.0,0.0,0.0,0.0,0.0,0.0
Mexico,0.0,0.0,0.0,0.0,3.0,0.0
U.S.,0.0,0.0,0.0,0.0,0.0,17.0


In [213]:
solution_year = [2018,2019,2020,2021,2022]
res_h_dict1 = {year:{id:pd.DataFrame() for id in all_exrate0.keys()} for year in solution_year}
res_r_dict1 = {year:{id:pd.DataFrame() for id in all_exrate0.keys()} for year in solution_year}

for id, exrate0 in tqdm(all_exrate0.items()):
    #print(f'solution for id:{id}')
    for year in solution_year:
        oph, opr = opt_main_func2(exrate0,year,0)
        res_h_dict1[year][id] = oph
        res_r_dict1[year][id] = opr

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.41it/s]


# Q3

In [174]:
best_h = sum(sum(res_h_dict1[year].values()) for year in solution_year)/500
dec_h = {(row, col): h.loc[row, col] for row in h.index for col in h.columns}

In [215]:
sum(sum(res_h_dict1[year].values()) for year in solution_year)/500

,LatinAmerica,Europe,AsiaWoJapan,Japan,Mexico,U.S.
from,,,,,,
Brazil,7.0,4.0,0.0,0.0,0.0,0.0
Germany,0.0,0.0,0.0,0.0,0.0,0.0
India,0.0,0.0,5.0,7.0,0.0,0.0
Japan,0.0,0.0,0.0,0.0,0.0,0.0
Mexico,0.0,11.0,0.0,0.0,3.0,13.0
U.S.,0.0,0.0,0.0,0.0,0.0,5.0


In [182]:
best_r = sum(sum(res_r_dict1[year].values()) for year in solution_year)/500
dec_r = {(row, col): r.loc[row, col] for row in r.index for col in r.columns}

In [361]:
sum(sum(res_r_dict1[year].values()) for year in solution_year)/500

,LatinAmerica,Europe,AsiaWoJapan,Japan,Mexico,U.S.
from,,,,,,
Brazil,7.0,0.0,0.0,0.0,0.0,0.0
Germany,0.0,12.0,0.0,5.6,0.6,3.4
India,0.0,0.0,3.0,2.4,0.0,0.0
Japan,0.0,0.0,0.0,0.0,0.0,0.0
Mexico,0.0,0.0,0.0,0.0,2.4,3.4
U.S.,0.0,0.0,0.0,0.0,0.0,10.2


In [198]:
# Define the fixed strategies
strategies = {
    "Conservative": {
        ('Brazil', 'Plant'): 1.0, ('Brazil', 'H'): 1.0, ('Brazil', 'R'): 1.0,
        ('Germany', 'Plant'): 0.0, ('Germany', 'H'): 0.0, ('Germany', 'R'): 0.0,
        ('India', 'Plant'): 1.0, ('India', 'H'): 1.0, ('India', 'R'): 1.0,
        ('Japan', 'Plant'): 0.0, ('Japan', 'H'): 0.0, ('Japan', 'R'): 0.0,
        ('Mexico', 'Plant'): 0.0, ('Mexico', 'H'): 0.0, ('Mexico', 'R'): 0.0,
        ('U.S.', 'Plant'): 1.0, ('U.S.', 'H'): 1.0, ('U.S.', 'R'): 1.0
    },
    "Diversified": {
        ('Brazil', 'Plant'): 1.0, ('Brazil', 'H'): 1.0, ('Brazil', 'R'): 1.0,
        ('Germany', 'Plant'): 1.0, ('Germany', 'H'): 0.0, ('Germany', 'R'): 1.0,
        ('India', 'Plant'): 1.0, ('India', 'H'): 1.0, ('India', 'R'): 1.0,
        ('Japan', 'Plant'): 0.0, ('Japan', 'H'): 0.0, ('Japan', 'R'): 0.0,
        ('Mexico', 'Plant'): 1.0, ('Mexico', 'H'): 1.0, ('Mexico', 'R'): 1.0,
        ('U.S.', 'Plant'): 1.0, ('U.S.', 'H'): 1.0, ('U.S.', 'R'): 1.0
    },
    "Flexible": {
        ('Brazil', 'Plant'): 1.0, ('Brazil', 'H'): 1.0, ('Brazil', 'R'): 1.0,
        ('Germany', 'Plant'): 1.0, ('Germany', 'H'): 0.0, ('Germany', 'R'): 1.0,
        ('India', 'Plant'): 1.0, ('India', 'H'): 1.0, ('India', 'R'): 1.0,
        ('Japan', 'Plant'): 1.0, ('Japan', 'H'): 0.0, ('Japan', 'R'): 1.0,
        ('Mexico', 'Plant'): 1.0, ('Mexico', 'H'): 1.0, ('Mexico', 'R'): 1.0,
        ('U.S.', 'Plant'): 1.0, ('U.S.', 'H'): 1.0, ('U.S.', 'R'): 0.0
    }
}

In [200]:
# As provided before. Paste it here again to remind me which function I use here
def calc_total_cost(dec_plant, dec_h, dec_r, base_yr=2019, selected_yr=2023, tariff=0):
    x_plant = np.array(list(dec_plant.values())).reshape(len(n_ctry), len(n_lines))
    x_h = np.array(list(dec_h.values())).reshape(len(n_ctry), len(n_ctry))
    x_r = np.array(list(dec_r.values())).reshape(len(n_ctry), len(n_ctry))

    # Adjust the cost using exchange rate of give year
    reindx = exrate.loc[:, f'{base_yr}'] / exrate.loc[:, f'{selected_yr}']

    pcosts_rev = pcosts.values * reindx.values.reshape(-1,1)
    pcosts_rev = pd.DataFrame(pcosts_rev, columns=pcosts.columns[0:], index=pcosts.index)
    # return pcosts_rev
    # pcosts_rev = adj_pcosts_exrate(2019, 2023)

    duties_mat = np.zeros(len(duties)) + (1 + duties['duty'].values)[:, np.newaxis]
    np.fill_diagonal(duties_mat, 1)
    duties_mat = pd.DataFrame(duties_mat.T, index=pcosts_rev.index, columns=duties.index)
    duties_mat.loc['Germany', 'U.S.'] += tariff
    duties_mat.loc['U.S.', 'Europe']  += tariff

    vcosts_h = tcosts.add(pcosts_rev['rm_h'], axis=0).add(pcosts_rev['pc_h'], axis=0) * duties_mat
    vcosts_r = tcosts.add(pcosts_rev['rm_r'], axis=0).add(pcosts_rev['pc_r'], axis=0) * duties_mat

    fc = pcosts_rev[['fc_p','fc_h','fc_r']].values
    vh = (vcosts_h * x_h).values
    vr = (vcosts_r * x_r).values
    total_cost = sum(0.2 * fc[i,j] for i in n_ctry for j in n_lines) + sum(0.8 * fc[i,j] * x_plant[i,j] for i in n_ctry for j in n_lines) + sum(vh[i,j] for i in n_ctry for j in n_ctry) + sum(vr[i,j] for i in n_ctry for j in n_ctry)

    return total_cost

In [204]:
# Draw 100 samples and evaluate each strategy
results = {}
for strategy_name, strategy_config in strategies.items():
    costs = []
    for i in range(100):
        exrate_sample = all_exrate0[i]
        cost = calc_total_cost(strategy_config, dec_h, dec_r, base_yr=2019, selected_yr=2023, tariff=0)
        costs.append(cost)

    results[strategy_name] = {
        "Expected Cost": np.mean(costs),
        "Cost Std Dev": np.std(costs)
    }

# Display the results
results_df = pd.DataFrame(results).T
print(results_df)

              Expected Cost  Cost Std Dev
Conservative    1172.239400  2.273737e-13
Diversified     1251.506118  2.273737e-13
Flexible        1260.522888  0.000000e+00
